In [2]:

# importing module
import sys
  
  
# printing all paths
sys.path

['e:\\Coding\\backtesters\\QuantFreedom\\tests',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python310\\DLLs',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python310\\lib',
 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python310',
 'e:\\Coding\\virtual environments\\qfFree',
 '',
 'e:\\Coding\\virtual environments\\qfFree\\lib\\site-packages',
 'e:\\coding\\backtesters\\quantfreedom',
 'e:\\Coding\\virtual environments\\qfFree\\lib\\site-packages\\win32',
 'e:\\Coding\\virtual environments\\qfFree\\lib\\site-packages\\win32\\lib',
 'e:\\Coding\\virtual environments\\qfFree\\lib\\site-packages\\Pythonwin']